## Meta-Summarization of IG Documents
This script aims to develop a prompt chain structure to send large amounts of text/content to LLM APIs through multiple calls. 

The current approach takes in all JSON files from the Plan Net IG, all figure diagrams, and key narrative information in markdown form (formerly extracted from HTML files). The script then summarizes each type of information in batches, and creates a meta-summarization of all documents to outline the technical information it can glean from all submitted documentation. The goal is to identify if this approach can produce all technical information at an appropriate level of deatil that an LLM would need to know to help design a test kit for a given IG. Best practices from the Claude API were used.

First attempts: We were able to run through the script fully using the Claude API with all JSONs, markdown content, and images. The process took over 93 minutes. The meta summary is saved in the file final_technical_analysis.md and is pasted at the end of this script. We can see that the first iteration did not produce detailed enough information about requirements, etc. 

In progress: 
- Doing a full run for Gemini and GPT, and instead of outputting summaries of the content, using this meta summarization process to produce list of test requirements, revising the prompting based on Inferno requirements extraction process documentation. 
- Reviewing LangChain iterative refinement capabilities to improve summary quality



### Script Organization:
1. Imports and Basic Setup
2. Configuration
3. File Processing Functions
4. Core Processing Functions
5. Batch Processing Functions
6. Verification and Query Functions
7. Main Execution Functions

### Setup

In [ ]:
# 1. IMPORTS AND BASIC SETUP
import base64
import json
import logging
from typing import List, Dict, Tuple, Union, Optional
from dataclasses import dataclass
import os
import time
import threading
from IPython.display import Image
import math
import io
import re
import pandas as pd
from json_repair import repair_json
from langchain_community.document_loaders import BSHTMLLoader
import shutil
from dotenv import load_dotenv
import httpx
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type
from anthropic import RateLimitError
from anthropic import Anthropic

# Basic setup
logging.basicConfig(level=logging.INFO)
load_dotenv()

# Constants
CERT_PATH = '/opt/homebrew/etc/openssl@3/cert.pem'


Read in API keys for Claude, Gemini, and GPT from .env file

In [ ]:
#gemini_api_key = os.getenv('GEMINI_API_KEY')
#OpenAI.api_key = os.getenv('OPENAI_API_KEY')

Set up Claude API

### Claude configuration

In [ ]:
# 2. CONFIGURATION
# Global configuration variables
SYSTEM_ROLE = """You are a seasoned Healthcare Integration Test Engineer with extensive FHIR experience 
that is looking to build a FHIR Test Kit for a specific Implementation Guide. You analyze technical documentation 
and requirements with a focus on testability, implementation verification, and conformance testing."""

CLAUDE_CONFIG = {
    "model_name": "claude-3-5-sonnet-20240620",
    "max_tokens": 8192,
    "requests_per_minute": 25,
    "delay_between_chunks": 2,
    "delay_between_batches": 10,
    "default_batch_size": 3
}

def create_claude_messages(prompt: str, assistant_prefix: str = None) -> list:
    """Helper function to create standardized message format"""
    return [
        {"role": "user", "content": prompt},
        {
            "role": "assistant", 
            "content": assistant_prefix or "As a Healthcare Integration Test Engineer, here is my analysis: "
        }
    ]

def create_claude_request(messages: list, stop_sequences: list = None) -> dict:
    """Helper function to create standardized request parameters"""
    return {
        "model": CLAUDE_CONFIG["model_name"],
        "messages": messages,
        "system": SYSTEM_ROLE,
        "max_tokens": CLAUDE_CONFIG["max_tokens"],
        "stop_sequences": stop_sequences
    }

# Client setup
def create_anthropic_client():
    """Create Anthropic client with proper certificate verification"""
    verify_path = CERT_PATH if os.path.exists(CERT_PATH) else True
    http_client = httpx.Client(
        verify=verify_path,
        timeout=30.0
    )
    return Anthropic(
        api_key=os.getenv('ANTHROPIC_API_KEY'),
        http_client=http_client
    )

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type((RateLimitError, TimeoutError))
)
def safe_claude_request(client, **kwargs):
    """Make a rate-limited request to Claude with retries"""
    rate_limiter.wait_if_needed()
    try:
        return client.messages.create(**kwargs)
    except Exception as e:
        logging.error(f"Error in Claude request: {str(e)}")
        raise

# Rate limiting setup
def create_rate_limiter(max_requests_per_minute=50):
    """Create a simple rate limiter"""
    class RateLimiter:
        def __init__(self):
            self.requests = []
            self.max_requests = max_requests_per_minute
            self.time_window = 60  # seconds

        def wait_if_needed(self):
            now = time.time()
            self.requests = [req_time for req_time in self.requests 
                           if now - req_time < self.time_window]
            
            if len(self.requests) >= self.max_requests:
                sleep_time = self.time_window - (now - self.requests[0])
                if sleep_time > 0:
                    time.sleep(sleep_time)
                self.requests = self.requests[1:]
            
            self.requests.append(now)
            
    return RateLimiter()

rate_limiter = create_rate_limiter(max_requests_per_minute=CLAUDE_CONFIG["requests_per_minute"])

### Pulling in files of interest
Sourcing JSON files from the IG, copying them from full-ig directory into json_only folder.

In [ ]:
# 3. FILE PROCESSING FUNCTIONS

def copy_json_files(source_folder='full-ig/site', destination_folder='full-ig/json_only'):
    """
    Copy JSON files from source to destination directory,
    excluding compound extensions and creating the directory if needed
    """
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    json_files = []
    for file_name in os.listdir(source_folder):
        # Check if the file ends with .json but not with compound extensions
        if (file_name.endswith('.json') and 
            not any(file_name.endswith(ext) for ext in [
                '.ttl.json', 
                '.jsonld.json', 
                '.xml.json', 
                '.change.history.json'
            ])):
            json_files.append(file_name)
            # Copy the file to the destination folder
            shutil.copy(os.path.join(source_folder, file_name), destination_folder)
            
    logging.info(f"Copied {len(json_files)} JSON files to {destination_folder}")
    return json_files

def group_files_by_base_name(directory_path: str, delimiter: str = '-') -> dict:
    """
    Group files in the directory by their base name.
    
    Args:
        directory_path: Path to the directory containing files
        delimiter: The delimiter to split the file name on
    Returns:
        Dictionary mapping base names to lists of related files
    """
    grouped_files = defaultdict(list)
    
    for filename in os.listdir(directory_path):
        if filename.endswith('.json') and delimiter in filename:
            base_name = filename.split(delimiter)[0]
            grouped_files[base_name].append(filename)
    
    return grouped_files

def copy_files_to_folders(directory_path: str, grouped_files: dict):
    """
    Create folders for each base name and copy related files into them.
    
    Args:
        directory_path: Path to the directory containing files
        grouped_files: Dictionary of grouped files by base name
    """
    for base_name, files in grouped_files.items():
        if len(files) >= 1:
            # Create base name folder
            base_folder = os.path.join(directory_path, base_name)
            if not os.path.exists(base_folder):
                os.makedirs(base_folder)
            logging.info(f"Created folder: {base_folder}")
            
            # Copy files to their respective folders
            for file in files:
                source_file = os.path.join(directory_path, file)
                destination_file = os.path.join(base_folder, file)
                shutil.copy(source_file, destination_file)

def consolidate_jsons(base_directory: str = 'full-ig/json_only'):
    """
    Consolidate related JSON files while maintaining object integrity.
    Creates combined files for each resource type.
    """
    subdirs = [d for d in os.listdir(base_directory) 
              if os.path.isdir(os.path.join(base_directory, d))]
    
    for subdir in subdirs:
        folder_path = os.path.join(base_directory, subdir)
        combined_data = []
        
        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                file_path = os.path.join(folder_path, filename)
                try:
                    with open(file_path, 'r') as f:
                        json_content = json.load(f)
                        if isinstance(json_content, dict) and 'entry' in json_content:
                            combined_data.extend(json_content['entry'])
                        else:
                            combined_data.append(json_content)
                except json.JSONDecodeError as e:
                    logging.error(f"Error decoding JSON from {filename}: {e}")
                    continue
        
        if combined_data:
            output_filename = f"{subdir}_combined.json"
            output_path = os.path.join(base_directory, output_filename)
            
            try:
                with open(output_path, 'w') as outfile:
                    json.dump({
                        "resourceType": subdir,
                        "total": len(combined_data),
                        "entry": combined_data
                    }, outfile, indent=2)
                logging.info(f"Created {output_filename} with {len(combined_data)} entries")
            except Exception as e:
                logging.error(f"Error writing {output_filename}: {e}")

def prepare_json_for_processing(json_file_path: str) -> Union[dict, list]:
    """
    Read and prepare JSON file for processing.
    Extracts entries if present or returns whole content.
    """
    with open(json_file_path, 'r') as f:
        data = json.load(f)
        
    if isinstance(data, dict) and 'entry' in data:
        return data['entry']
    return data

def split_json(json_data: Union[dict, list], max_size: int = 2000) -> List[list]:
    """
    Split JSON array into chunks while maintaining complete JSON objects.
    
    Args:
        json_data: JSON data to split
        max_size: Maximum size for each chunk in characters
    Returns:
        List of chunks, where each chunk contains complete JSON objects
    """
    if isinstance(json_data, dict):
        json_data = [json_data]
    
    chunks = []
    current_chunk = []
    current_size = 0
    
    for item in json_data:
        item_size = len(json.dumps(item))
        
        # Handle large individual items
        if item_size > max_size:
            if current_chunk:
                chunks.append(current_chunk)
                current_chunk = []
                current_size = 0
            chunks.append([item])
            continue
        
        # Start new chunk if current would exceed max_size
        if current_size + item_size > max_size and current_chunk:
            chunks.append(current_chunk)
            current_chunk = []
            current_size = 0
        
        current_chunk.append(item)
        current_size += item_size
    
    if current_chunk:
        chunks.append(current_chunk)
        
    return chunks

def encode_image(image_path: str) -> str:
    """Convert image to base64 encoding for API consumption"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def clean_markdown(text: str) -> str:
    """Clean markdown content by removing unnecessary whitespace and formatting"""
    # Remove multiple newlines
    text = re.sub(r'\n\s*\n', '\n\n', text)
    
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # Remove excessive punctuation 
    text = re.sub(r'\.{2,}', '.', text)
    
    # Remove escaped characters
    text = re.sub(r'\\(.)', r'\1', text)
    
    # Remove table formatting but keep content
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'[-\s]*\n[-\s]*', '\n', text)
    
    return text.strip()

### Preparing Files for LLM

1. Because we have so many JSONs, we cannot feed them all to an LLM at once. These functions split combined JSONs into chunks that can fit in one LLM call. Each chunk only contains full JSONs so individual JSONs are not split up. We included this feature to try and maintain summarization quality- the LLM should receive all relevant information together instead of in pieces, to help it understand what it is receiving.

2. We also convert images to base 64 encoding so they can be read by an LLM.

3. We set up a prompt for summarizing JSON chunks.

In [ ]:
# 4. CORE PROCESSING FUNCTIONS

def create_json_summary_prompt(chunk: Union[dict, list], chunk_num: int, total_chunks: int) -> str:
    """
    Create standardized prompt for summarizing JSON chunk with test engineering focus.
    
    Args:
        chunk: The JSON chunk to analyze
        chunk_num: Current chunk number
        total_chunks: Total number of chunks
    """
    return f"""Analyze this portion ({chunk_num} of {total_chunks}) of a FHIR Implementation Guide JSON resource bundle.
    Focus on key technical details, requirements, and relationships that need testing.
    
    JSON Content:
    {json.dumps(chunk, indent=2)}
    
    Please provide:
    1. An overview of what is included in the group of JSONs and what piece of the IG they represent
    2. Resource Types and Profiles present that will need test coverage
    3. Key technical requirements and constraints that must be validated
    4. Dependencies and relationships between resources that need testing
    5. Specific conformance requirements that need verification
    
    Focus on new information not covered in previous chunks."""

def process_json_file(client: Anthropic, json_file_path: str) -> str:
    """
    Process a JSON file while maintaining object integrity.
    Splits into chunks if needed and processes each chunk.
    """
    try:
        json_data = prepare_json_for_processing(json_file_path)
        chunks = split_json(json_data)
        chunk_summaries = []
        
        for i, chunk in enumerate(chunks):
            prompt = create_json_summary_prompt(chunk, i+1, len(chunks))
            
            messages = create_claude_messages(
                prompt=prompt,
                assistant_prefix="Here is the technical summary with testing implications: <summary>"
            )
            
            kwargs = create_claude_request(
                messages=messages,
                stop_sequences=["</summary>"]
            )
            
            response = safe_claude_request(client, **kwargs)
            chunk_summaries.append(response.content[0].text)
            time.sleep(CLAUDE_CONFIG["delay_between_chunks"])
        
        return combine_summaries(client, chunk_summaries)
    except Exception as e:
        logging.error(f"Error processing file {json_file_path}: {str(e)}")
        return f"Error processing file: {str(e)}"

def process_markdown(client: Anthropic, content: str) -> str:
    """
    Process markdown content and generate a technical summary focused on testing requirements.
    """
    try:
        prompt = f"""Analyze this technical documentation markdown content from a testing perspective:

        {content}

        Please provide:
        1. Key technical concepts and definitions that need validation
        2. Important requirements and specifications that must be tested
        3. Technical workflows or processes that require test coverage
        4. Dependencies or prerequisites that need verification
        5. Implementation details and guidelines that should be validated
        
        Focus on extracting testable requirements and validation criteria."""
        
        messages = create_claude_messages(
            prompt=prompt,
            assistant_prefix="Here is the technical summary with testing implications: <summary>"
        )
        
        kwargs = create_claude_request(
            messages=messages,
            stop_sequences=["</summary>"]
        )
        
        response = safe_claude_request(client, **kwargs)
        return response.content[0].text
    except Exception as e:
        logging.error(f"Error processing markdown content: {str(e)}")
        return "Error processing markdown content: " + str(e)

def process_image(client: Anthropic, image_path: str) -> str:
    """
    Process a single image and generate a technical description focused on testing implications.
    """
    try:
        base64_image = encode_image(image_path)
        
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "source": {
                            "type": "base64",
                            "media_type": "image/" + image_path.split('.')[-1],
                            "data": base64_image
                        }
                    },
                    {
                        "type": "text",
                        "text": """Analyze this technical diagram/figure from a testing perspective. Focus on:
                        1. Key components and their relationships that need validation
                        2. Technical workflows or processes that require test coverage
                        3. Architecture or design patterns that need verification
                        4. Integration points that must be tested
                        5. Important technical details or annotations that affect testing
                        Provide a detailed technical description with testing implications."""
                    }
                ]
            }
        ]
        
        kwargs = create_claude_request(
            messages=messages,
            stop_sequences=["</summary>"]
        )
        
        response = safe_claude_request(client, **kwargs)
        return response.content[0].text
        
    except Exception as e:
        logging.error(f"Error processing image {image_path}: {str(e)}")
        raise

def combine_summaries(client: Anthropic, summaries: List[str]) -> str:
    """
    Combine chunk summaries into a cohesive analysis focused on testing requirements.
    """
    try:
        prompt = f"""Synthesize these related summaries into a unified technical analysis 
        focused on testing requirements:

        {json.dumps(summaries, indent=2)}
        
        Create a comprehensive analysis that:
        1. Eliminates redundant information
        2. Maintains technical accuracy
        3. Includes specific technical information about:
           - Search parameters that need testing
           - Resource profiles that require validation
           - Must Support elements that need verification
           - Conformance requirements that must be tested
        4. Preserves important relationships and workflows that need test coverage"""
        
        messages = create_claude_messages(
            prompt=prompt,
            assistant_prefix="Here is the combined analysis with testing implications: <summary>"
        )
        
        kwargs = create_claude_request(
            messages=messages,
            stop_sequences=["</summary>"]
        )
        
        response = safe_claude_request(client, **kwargs)
        return response.content[0].text
    except Exception as e:
        logging.error(f"Error combining summaries: {str(e)}")
        return "Unable to combine summaries due to error"

def create_meta_summary(client: Anthropic, 
                       json_summaries: Dict[str, str], 
                       markdown_summaries: Dict[str, str], 
                       image_summaries: Dict[str, str]) -> str:
    """
    Create a comprehensive meta-summary focused on testing requirements.
    """
    try:
        prompt = f"""Synthesize information from multiple content types into a comprehensive 
        technical analysis focused on testing requirements:

        JSON Configuration Summaries:
        {json.dumps(json_summaries, indent=2)}

        Documentation Summaries:
        {json.dumps(markdown_summaries, indent=2)}

        Diagram/Figure Analyses:
        {json.dumps(image_summaries, indent=2)}

        Create a comprehensive technical analysis that outlines:
        1. Technical Requirements and Architecture
           - Core technical requirements that need testing
           - System architecture components requiring validation
           - Integration points needing test coverage
           
        2. Implementation Details
           - Key configurations requiring verification
           - Resource profiles needing validation
           - Constraints and rules requiring testing
           
        3. Testing Implications
           - Required test scenarios
           - Validation approaches
           - Conformance verification methods
           
        4. Culminating Analysis
           - Overall testing strategy
           - Key risk areas requiring thorough testing
           - Implementation considerations for test development"""

        messages = create_claude_messages(
            prompt=prompt,
            assistant_prefix="Here is the comprehensive technical analysis for test development: <summary>"
        )
        
        kwargs = create_claude_request(
            messages=messages,
            stop_sequences=["</summary>"]
        )
        
        response = safe_claude_request(client, **kwargs)
        return response.content[0].text
    except Exception as e:
        logging.error(f"Error creating meta-summary: {str(e)}")
        return "Unable to create meta-summary due to error"

#### Defining Rate Limiting & Safe Call Functions

Because of the amount of content we are sending to APIs, we need to include rate limiting in our prompt chaining process to avoid hitting rate limits. This includes a function to create a reate limiter and to make calls to the Claude LLM with the rate limiter included. 

#### Setting up Processing Functions
This set of functions allows for summarizing of batches of each information type (e.g., JSONs, markdown, and images) from individual files, combining those summaries at the directory level/file type level, and then sending those combined summaries to the LLM at once to ask for one meta-summarization. The meta-summary is saved to an output file for review.

In [ ]:
# 5. BATCH PROCESSING FUNCTIONS

def process_content(client: Anthropic, file_path: str, content_type: str) -> str:
    """
    Generic content processor that handles any content type.
    
    Args:
        client: Anthropic client instance
        file_path: Path to the file to process
        content_type: Type of content ('json', 'markdown', or 'image')
    Returns:
        Processed content summary
    """
    try:
        if content_type == "json":
            return process_json_file(client, file_path)
        elif content_type == "markdown":
            with open(file_path, 'r') as f:
                content = clean_markdown(f.read())
            return process_markdown(client, content)
        elif content_type == "image":
            return process_image(client, file_path)
        else:
            raise ValueError(f"Unsupported content type: {content_type}")
            
    except Exception as e:
        logging.error(f"Error processing {content_type} file {file_path}: {str(e)}")
        return f"Error processing file: {str(e)}"

def process_batch(client: Anthropic, 
                 files: List[str], 
                 content_type: str) -> Dict[str, str]:
    """
    Process a batch of files with rate limiting.
    
    Args:
        client: Anthropic client instance
        files: List of file paths to process
        content_type: Type of content ('json', 'markdown', or 'image')
    Returns:
        Dictionary mapping file paths to their processed summaries
    """
    results = {}
    for i in range(0, len(files), CLAUDE_CONFIG["default_batch_size"]):
        batch = files[i:i + CLAUDE_CONFIG["default_batch_size"]]
        logging.info(f"Processing batch {i//CLAUDE_CONFIG['default_batch_size'] + 1} of {math.ceil(len(files)/CLAUDE_CONFIG['default_batch_size'])}")
        
        for file in batch:
            try:
                logging.info(f"Processing {content_type} file: {os.path.basename(file)}")
                results[file] = process_content(client, file, content_type)
            except Exception as e:
                logging.error(f"Error processing {file}: {str(e)}")
                results[file] = f"Error: {str(e)}"
                
        if len(batch) < CLAUDE_CONFIG["default_batch_size"]:
            logging.info("Processed final partial batch")
        time.sleep(CLAUDE_CONFIG["delay_between_batches"])
    
    return results

def process_all_content(client: Anthropic, base_directory: str = 'full-ig') -> str:
    """
    Process all content types using unified batch processing with rate limiting.
    
    Args:
        client: Anthropic client instance
        base_directory: Base directory containing all content
    Returns:
        Meta-summary of all processed content
    """
    try:
        # Process JSONs
        logging.info("Processing JSON files...")
        json_files = [
            os.path.join(base_directory, 'json_only', f) 
            for f in os.listdir(os.path.join(base_directory, 'json_only')) 
            if f.endswith('_combined.json')
        ]
        json_summaries = process_batch(client, json_files, "json")
        logging.info(f"Processed {len(json_files)} JSON files")
        
        # Process markdown files
        markdown_summaries = {}
        markdown_dir = os.path.join(base_directory, 'markdown')
        if os.path.exists(markdown_dir):
            logging.info("Processing markdown files...")
            md_files = [
                os.path.join(markdown_dir, f) 
                for f in os.listdir(markdown_dir) 
                if f.endswith('.md')
            ]
            markdown_summaries = process_batch(client, md_files, "markdown")
            logging.info(f"Processed {len(md_files)} markdown files")
        
        # Process images
        image_summaries = {}
        image_dir = os.path.join(base_directory, 'site/Figures')
        if os.path.exists(image_dir):
            logging.info("Processing image files...")
            img_files = [
                os.path.join(image_dir, f) 
                for f in os.listdir(image_dir) 
                if f.lower().endswith(('.png', '.jpg', '.jpeg'))
            ]
            image_summaries = process_batch(client, img_files, "image")
            logging.info(f"Processed {len(img_files)} image files")
        
        logging.info("Creating meta-summary...")
        time.sleep(CLAUDE_CONFIG["delay_between_batches"])  # Extra delay before meta-summary
        
        return create_meta_summary(
            client, 
            json_summaries, 
            markdown_summaries, 
            image_summaries
        )
        
    except Exception as e:
        logging.error(f"Error processing content: {str(e)}")
        raise

def save_processed_content(base_directory: str = 'full-ig', 
                         output_directory: str = 'processed_output') -> Optional[str]:
    """
    Save all processed content with progress tracking.
    
    Args:
        base_directory: Base directory containing all content
        output_directory: Directory to save processed content
    Returns:
        Final technical analysis or None if error occurs
    """
    os.makedirs(output_directory, exist_ok=True)
    
    client = create_anthropic_client()
    
    try:
        logging.info("Starting content processing...")
        final_summary = process_all_content(client, base_directory)
        
        output_file = os.path.join(output_directory, 'final_technical_analysis.md')
        with open(output_file, 'w') as f:
            f.write(final_summary)
        
        logging.info(f"Processing complete. Results saved to {output_file}")
        return final_summary
    
    except Exception as e:
        logging.error(f"Processing failed: {str(e)}")
        print(f"Error during processing: {str(e)}")
        return None

### Querying LLM after Passing through all content

In [ ]:
# 6. VERIFICATION AND QUERY FUNCTIONS

def create_verification_questions() -> List[str]:
    """
    Create a standardized list of verification questions focused on test kit development.
    Returns a list of questions to verify understanding of the IG.
    """
    return [
        "What is the purpose of the FHIR DaVinci PlanNet Implementation Guide?",
        "Who are the intended users and actors of the FHIR DaVinci PlanNet Implementation Guide?",
        "Are there one or more workflows defined in the FHIR DaVinci PDex Plan Net Implementation Guide? Please use all the information you know",
        "What data is being exchanged in the FHIR DaVinci PDex Plan Net Implementation Guide and why?",
        "How is that data represented by the resources and profiles in the FHIR DaVinci PDex Plan Net Implementation Guide? Create a list of the CRUDs + search parameters, create a code skeleton that would test each of the items in the list and then write the code",
        "What actions (REST/CRUD) or operations can be used in the FHIR DaVinci PDex Plan Net Implementation Guide?",
        "What are all the mandatory requirements and rules from the DaVinci PDex Plan Net Implementation Guide for compliant implementations? Which ones are fulfilled from the code you wrote to test those items from the last question?",
        "What are all the optional requirements and rules from the DaVinci PDex Plan Net Implementation Guide for compliant implementations?",
        "Create a test plan for the FHIR DaVinci RDcx Plan Net Implementation Guide"
    ]

def process_query(client: Anthropic,
                 question: str, 
                 context: str, 
                 conversation_history: Optional[List[Tuple[str, str]]] = None) -> str:
    """
    Process a single query about the technical content with conversation context.
    
    Args:
        client: Anthropic client instance
        question: The question to ask
        context: Technical analysis context
        conversation_history: Optional list of previous Q&A pairs
    Returns:
        Claude's response
    """
    try:
        prompt = f"""{SYSTEM_ROLE}

        Technical Analysis Context:
        {context}

        Question: {question}"""
        
        if conversation_history:
            prompt += "\n\nPrevious relevant discussion:\n"
            for q, a in conversation_history[-3:]:  # Keep last 3 exchanges
                prompt += f"\nQ: {q}\nA: {a}\n"
        
        messages = create_claude_messages(prompt)
        kwargs = create_claude_request(messages)
        
        response = safe_claude_request(client, **kwargs)
        return response.content[0].text
        
    except Exception as e:
        logging.error(f"Error processing query: {str(e)}")
        raise

def run_verification_analysis(client: Anthropic,
                            context_file: str,
                            output_dir: str) -> Dict[str, str]:
    """
    Run verification questions on the processed technical analysis.
    
    Args:
        client: Anthropic client instance
        context_file: Path to the technical analysis file
        output_dir: Directory to save verification results
    Returns:
        Dictionary of questions and answers
    """
    try:
        # Load the technical analysis
        with open(context_file, 'r') as f:
            context = f.read()
        
        # Process verification questions
        conversation_history = []
        verification_results = {}
        
        logging.info("Starting verification questions analysis...")
        questions = create_verification_questions()
        
        for i, question in enumerate(questions, 1):
            logging.info(f"Processing question {i} of {len(questions)}: {question[:100]}...")
            
            answer = process_query(
                client=client,
                question=question,
                context=context,
                conversation_history=conversation_history
            )
            
            verification_results[question] = answer
            conversation_history.append((question, answer))
            logging.info(f"Answer received: {answer[:200]}...")
            
            time.sleep(CLAUDE_CONFIG["delay_between_chunks"])
        
        # Save verification results
        verification_output = os.path.join(output_dir, 'verification_results.md')
        with open(verification_output, 'w') as f:
            f.write("# Implementation Guide Test Kit Development Analysis\n\n")
            for question, answer in verification_results.items():
                f.write(f"## Question\n{question}\n\n")
                f.write(f"## Test Engineering Analysis\n{answer}\n\n")
                f.write("---\n\n")
        
        logging.info(f"Verification analysis complete. Results saved to {verification_output}")
        return verification_results
        
    except Exception as e:
        logging.error(f"Error during verification analysis: {str(e)}")
        raise

def ask_interactive_question(client: Anthropic,
                           question: str,
                           context_file: str,
                           conversation_history: Optional[List[Tuple[str, str]]] = None) -> Optional[str]:
    """
    Interactive function to ask additional questions about the IG.
    
    Args:
        client: Anthropic client instance
        question: Question to ask
        context_file: Path to the technical analysis file
        conversation_history: Optional list of previous Q&A pairs
    Returns:
        Answer to the question or None if error occurs
    """
    try:
        # Load the technical analysis
        with open(context_file, 'r') as f:
            context = f.read()
        
        answer = process_query(
            client=client,
            question=question,
            context=context,
            conversation_history=conversation_history
        )
        
        print(f"\nQ: {question}")
        print(f"\nA: {answer}")
        return answer
        
    except Exception as e:
        logging.error(f"Error processing question: {str(e)}")
        print(f"Error processing question: {str(e)}")
        return None

class InteractiveQuerySession:
    """Class to maintain state for an interactive query session"""
    
    def __init__(self, client: Anthropic, context_file: str):
        self.client = client
        self.context_file = context_file
        self.conversation_history: List[Tuple[str, str]] = []
    
    def ask_question(self, question: str) -> Optional[str]:
        """Ask a question and maintain conversation history"""
        answer = ask_interactive_question(
            self.client,
            question,
            self.context_file,
            self.conversation_history
        )
        
        if answer:
            self.conversation_history.append((question, answer))
        
        return answer
    
    def save_conversation(self, output_file: str):
        """Save the conversation history to a file"""
        with open(output_file, 'w') as f:
            f.write("# Interactive Query Session\n\n")
            for question, answer in self.conversation_history:
                f.write(f"## Question\n{question}\n\n")
                f.write(f"## Answer\n{answer}\n\n")
                f.write("---\n\n")

In [ ]:
# Create an interactive query session
session = InteractiveQuerySession(client, 'summarized_output/technical_summary1.md')

# Ask questions
session.ask_question("What specific test cases should be created for the Location resource?")
session.ask_question("How should we validate the search parameters?")

# Save the conversation
session.save_conversation('summarized_output/query_session.md')

In [ ]:
# 7. MAIN EXECUTION FUNCTIONS

def initialize_environment(base_directory: str = 'full-ig',
                         output_directory: str = 'processed_output') -> Tuple[Anthropic, str, str]:
    """
    Initialize the processing environment.
    
    Args:
        base_directory: Base directory for input files
        output_directory: Directory for output files
    Returns:
        Tuple of (client, base_directory, output_directory)
    """
    # Setup logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s'
    )
    
    # Create output directory
    os.makedirs(output_directory, exist_ok=True)
    
    # Initialize client
    client = create_anthropic_client()
    
    # Create rate limiter
    rate_limiter = create_rate_limiter(
        max_requests_per_minute=CLAUDE_CONFIG["requests_per_minute"]
    )
    
    return client, base_directory, output_directory

def run_initial_analysis(client: Anthropic,
                        base_directory: str,
                        output_directory: str) -> Optional[str]:
    """
    Run the initial analysis phase including file processing and meta-summary.
    
    Args:
        client: Anthropic client instance
        base_directory: Base directory for input files
        output_directory: Directory for output files
    Returns:
        Path to the technical analysis file or None if error occurs
    """
    try:
        logging.info("Starting initial analysis phase...")
        
        # Process and organize files
        logging.info("Organizing files...")
        copied_files = copy_json_files()
        grouped_files = group_files_by_base_name('full-ig/json_only')
        copy_files_to_folders('full-ig/json_only', grouped_files)
        consolidate_jsons('full-ig/json_only')
        
        # Generate technical analysis
        logging.info("Generating technical analysis...")
        final_analysis = process_all_content(client, base_directory)
        
        # Save technical analysis
        analysis_file = os.path.join(output_directory, 'technical_summary1.md')
        with open(analysis_file, 'w') as f:
            f.write(final_analysis)
        
        logging.info(f"Initial analysis complete. Results saved to {analysis_file}")
        return analysis_file
        
    except Exception as e:
        logging.error(f"Error during initial analysis: {str(e)}")
        return None

def run_verification_phase(client: Anthropic,
                         analysis_file: str,
                         output_directory: str) -> Optional[Dict[str, str]]:
    """
    Run the verification phase using the completed technical analysis.
    
    Args:
        client: Anthropic client instance
        analysis_file: Path to the technical analysis file
        output_directory: Directory for output files
    Returns:
        Dictionary of verification results or None if error occurs
    """
    try:
        logging.info("Starting verification phase...")
        verification_results = run_verification_analysis(
            client,
            analysis_file,
            output_directory
        )
        logging.info("Verification phase complete")
        return verification_results
        
    except Exception as e:
        logging.error(f"Error during verification phase: {str(e)}")
        return None

def setup_interactive_session(client: Anthropic,
                            analysis_file: str) -> InteractiveQuerySession:
    """
    Set up an interactive query session for additional questions.
    
    Args:
        client: Anthropic client instance
        analysis_file: Path to the technical analysis file
    Returns:
        Configured InteractiveQuerySession instance
    """
    return InteractiveQuerySession(client, analysis_file)

def run_full_analysis(base_directory: str = 'full-ig',
                     output_directory: str = 'processed_output') -> Dict[str, Any]:
    """
    Run the complete analysis pipeline including verification.
    
    Args:
        base_directory: Base directory for input files
        output_directory: Directory for output files
    Returns:
        Dictionary containing results and session objects
    """
    try:
        # Initialize environment
        client, base_dir, output_dir = initialize_environment(
            base_directory,
            output_directory
        )
        
        # Run initial analysis
        analysis_file = run_initial_analysis(client, base_dir, output_dir)
        if not analysis_file:
            raise RuntimeError("Initial analysis failed")
            
        # Run verification
        verification_results = run_verification_phase(
            client,
            analysis_file,
            output_dir
        )
        
        # Setup interactive session
        query_session = setup_interactive_session(client, analysis_file)
        
        return {
            'analysis_file': analysis_file,
            'verification_results': verification_results,
            'query_session': query_session,
            'output_directory': output_dir
        }
        
    except Exception as e:
        logging.error(f"Error during full analysis: {str(e)}")
        raise




In [ ]:
# Run full analysis
results = run_full_analysis()

# Print summary
print("\nAnalysis Complete!")
print(f"Technical analysis saved to: {results['analysis_file']}")
print(f"Verification results available in: {results['output_directory']}")
print("\nYou can now use the query session for additional questions:")
print("query_session = results['query_session']")
print("answer = query_session.ask_question('Your question here')")

In [ ]:
# Access the interactive session
session = results['query_session']

In [ ]:
# Ask additional questions
answer = session.ask_question("What are the key test scenarios for the Location resource?")

In [ ]:
# Save the conversation
session.save_conversation('summarized_output/interactive_session.md')